Reference：Evolutionary dimension reduction in phenotypic space

DOI: 10.1103/PhysRevResearch.2.013197

In [ ]:
###cell block for functions

import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from numpy import savetxt
from sklearn.decomposition import PCA

N= 400 #化学物質の数
n = 10     #nutrient・transporterの数
D = 1/N #拡散係数
rho = 0.04   #各化学物質あたりの反応経路の数
L = 10 #各世代で選ばれる細胞数
c = 10 #コピー数
myu = 0.04 #突然変異率
G = 3000
#世代数
div = 50 #saveする間隔
M = 3000
#if D is too small, the evolution is not fast...coz nutrient is not supplied with sufficient amount
#Is mutation rate too small?
#Is rho too small?
#(23:05) When rho and myu become big and the input has 1 in the half of n, the evolution had went well 

dt=0.1             
t_max=500       
times=np.arange(0,t_max,dt)

environment = np.array([1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]) / 5

def generate_genotype(c, L, N, rho,trial_number):
  genotype_1 = np.zeros((c*L, N, int(N*rho)))           #reactant
  genotype_2 = np.zeros((c*L, N, int(N*rho)))            #product
  for a in range(c*L):
    for k in range(N):                              #index for catalyst
      for num in range(int(N*rho)):
        genotype_1[a, k, num] = int(np.random.rand() * N)      #index for reactant
        genotype_2[a, k, num] = int(np.random.rand() * N)      #index for product
  savetxt(str(trial_number)+"genotype_1.csv", genotype_1, delimiter = ",")
  savetxt(str(trial_number)+"genotype_2.csv", genotype_2, delimiter = ",")
  return genotype_1, genotype_2
    
def mutate_network(parents_1, parents_2, N, c, L, myu):
  children_1 = np.zeros((c*L,N,int(N*rho)))
  children_2 = np.zeros((c*L,N,int(N*rho)))
  change_genotype_1 = []
  change_genotype_2 = []
  

  for a in range(c*L):
    if a < L:
      children_1[a,:,:] = parents_1[a,:,:]      #copy the parents
      children_2[a,:,:] = parents_2[a,:,:]      #copy the parents
    else:
        for b in range(int(N*myu)):
            t = a % L
            indices = [[],[]]
            children_1[a,:,:] = parents_1[t,:,:]    #copy the parents, making the basis of mutation
            children_2[a,:,:] = parents_2[t,:,:]    #copy the parents, making the basis of mutation
            
            #pick two reaction (i,j,k) and (l,m,n) by selecting the indices of the genotypes
            c = int(np.random.rand() * N * rho) #get cth reaction number from genotype[t,x,c]
            x = int(np.random.rand() * N)
            d = int(np.random.rand() * N * rho)
            y = int(np.random.rand() * N)

            i = children_1[a, x, c]
            j = children_2[a, x, c]
            k = x
            l = children_1[a, y, d]
            m = children_2[a, y, d]
            n = y

            if i != m and l != m and m != k and j != n:
                if np.random.rand() < 0.5:
                    children_2[a, x, c] = m
                    children_2[a, y, d] = j
                    #print([x,y,c,d])
                    change_genotype_2.append([x,y,c,d])   #since there are c*L copies, the data strucure will be (c*L, some_random_number, 2)
                                                                    #don't forget to point out the first indices
                    #print("change_genotype_2 is", change_genotype_2)
                else:
                    children_1[a, x, c] = l
                    children_1[a, y, d] = i
                    change_genotype_1.append([x,y,c,d])
                    #print("change_genotype_1 is", change_genotype_1)

  #print("mutate_network is done")
  return children_1, children_2, change_genotype_1, change_genotype_2

##mutate network but the mutation involving 4 and 5 is excluded
def mutate_network2(parents_1, parents_2, N, c, L, myu):
  important_chemical = [4,5]
  children_1 = np.zeros((c*L,N,int(N*rho)))
  children_2 = np.zeros((c*L,N,int(N*rho)))
  change_genotype_1 = []
  change_genotype_2 = []
  

  for a in range(c*L):
    if a < L:
      children_1[a,:,:] = parents_1[a,:,:]      #copy the parents
      children_2[a,:,:] = parents_2[a,:,:]      #copy the parents
    else:
        counter = 0  #N*myu回処理をしたいから、0から始めて、N*myu-1になるまでやることにした
        while counter < int(N*myu):
            for b in range(int(N*myu)):
                t = a % L
                indices = [[],[]]
                children_1[a,:,:] = parents_1[t,:,:]    #copy the parents, making the basis of mutation
                children_2[a,:,:] = parents_2[t,:,:]    #copy the parents, making the basis of mutation

                c = int(np.random.rand() * N * rho) #get cth reaction number from genotype[t,x,c]
                x = int(np.random.rand() * N)
                d = int(np.random.rand() * N * rho)
                y = int(np.random.rand() * N)

                i = children_1[a, x, c]
                j = children_2[a, x, c]
                k = x
                l = children_1[a, y, d]
                m = children_2[a, y, d]
                n = y

                c_1 = (i != m and l != m and m != k and j != n)

                list_1 = [i,j,k,l,m,n]
                c_2 = True
                for i in range(len(important_chemical)):
                    c_ = (list_1.count(important_chemical[i]) == 0)
                    c_2 = (c_2) and c_


                if c_1 and c_2:
                    if np.random.rand() < 0.5:
                        children_2[a, x, c] = m
                        children_2[a, y, d] = j
                        #print([x,y,c,d])
                        change_genotype_2.append([x,y,c,d])   #since there are c*L copies, the data strucure will be (c*L, some_random_number, 2)
                                                                        #don't forget to point out the first indices
                        counter += 1

                    #print("change_genotype_2 is", change_genotype_2)
                    else:
                        children_1[a, x, c] = l
                        children_1[a, y, d] = i
                        change_genotype_1.append([x,y,c,d])
                        #print("change_genotype_1 is", change_genotype_1)
                        counter += 1

  #print("mutate_network is done")
  return children_1, children_2, change_genotype_1, change_genotype_2

#a function to generate random_string for  get_dx_2: get_dx_2 用にrandom_stringを生成する関数
def mutate_for_type1(number):
    random_string = []
    string_for_save = []
    for i in range(int(number)):
        r_num = int(N*rho*np.random.rand())
        r_k = int(N * np.random.rand())
        random_string.append(r_num * N + r_k)
        string_for_save.append([r_num, r_k])
    #print(random_string)
    return random_string, string_for_save
#here, the data structure of string_for_save is (number, 2)


#a function to restore the mutated genotype from the original genotype and the list of changes
def restore_genotype(single_genotype_1, single_genotype_2, change_single_genotype_1, change_single_genotype_2,check):
    #c_single_genotype_1 = single_genotype_1
    #c_single_genotype_2 = single_genotype_2
    #print(check,"th original genotype input  in restore is", single_genotype_1)
    change_single_genotype_1 = change_single_genotype_1[~(change_single_genotype_1 == 400)]
    change_single_genotype_2 = change_single_genotype_2[~(change_single_genotype_2 == 400)]
    
    mutated_single_genotype_1 = np.zeros((N,int(N*rho)))
    mutated_single_genotype_2 = np.zeros((N,int(N*rho)))
    
    for i in range(N):
            for j in range(int(N*rho)):
                mutated_single_genotype_1[i,j] = single_genotype_1[i,j]
    for i in range(N):
            for j in range(int(N*rho)):
                mutated_single_genotype_2[i,j] = single_genotype_2[i,j]
    
    #print(check,"th original genotype input  in middle 1 is", single_genotype_1)
    for a in range(len(change_single_genotype_1)//4 - 1):
        
        [x,y,c,d] = change_single_genotype_1[4*a:4*a+4]
        
        [x,y,c,d] = [int(x),int(y),int(c),int(d)]
        l = single_genotype_1[x, c]
        i = single_genotype_1[y, d]
        mutated_single_genotype_1[x, c] = i   #ここで誤作動起こしてる
        mutated_single_genotype_1[y, d] = l
    #print(check,"th original genotype input  in middle 2 is", single_genotype_1)
        
    for b in range(len(change_single_genotype_2)//4 - 1):
        mutated_single_genotype_2 = single_genotype_2
        [x,y,c,d] = change_single_genotype_2[4*b:4*b+4]
        [x,y,c,d] = [int(x),int(y),int(c),int(d)]
        l = single_genotype_2[x, c]
        i = single_genotype_2[y, d]
        mutated_single_genotype_2[x, c] = i
        mutated_single_genotype_2[y, d] = l
    #print(check,"th original genotype input in restore at the end is", single_genotype_1)
    print(mutated_single_genotype_1)
    if check == 0:
        savetxt("original_genotype_1??.csv", mutated_single_genotype_1, delimiter = ",")
    return mutated_single_genotype_1, mutated_single_genotype_2

###generate a string of numbers that corresponds to the number of mutations involving 4 or 5
def detect_mut(single_genotype_1, single_genotype_2, change_genotype_1, change_genotype_2):
    mut_counter = [0]     #counter for the first genotype(the original)
    #print("original genotype is", single_genotype_1)
    for i in range(len(change_genotype_1)):
        c_single_genotype_1 = single_genotype_1
        c_single_genotype_2 = single_genotype_2
        change_single_genotype_1 =  change_genotype_1[i]
        #print(change_single_genotype_1)
        change_single_genotype_2 =  change_genotype_2[i]
        #print(c_single_genotype_1)
        mutated_single_genotype_1, mutated_single_genotype_2 = restore_genotype(single_genotype_1=c_single_genotype_1, single_genotype_2=c_single_genotype_2, change_single_genotype_1=change_single_genotype_1,  change_single_genotype_2=change_single_genotype_2,check=i)
        #print("mutated_single_genotype is", mutated_single_genotype_1)
        #print("original genotype is", c_single_genotype_1)
        mut = 0
        for a in range(N):
            for b in range(int(N*rho)):
                shift_1 = mutated_single_genotype_1[a,b] - c_single_genotype_1[a,b]
                #if (a == 2) and (b == 2):
                    #print("mutated_single_genotype_1[2,2]=", mutated_single_genotype_1[2,2])
                    #print("single_genotype_1[2,2]=", single_genotype_1[2,2])
                #print(shift_1)
                #if shift_1 != 0:
                    #print("shift_1 != 0")
                shift_2 = mutated_single_genotype_2[a,b] - c_single_genotype_2[a,b]
                if (shift_1 != 0) or (shift_2 != 0):
                    i_4 = (c_single_genotype_1[a,b] == 4)
                    j_4 = (c_single_genotype_2[a,b] == 4)
                    k_4 = (a == 4)
                    i_5 = (c_single_genotype_1[a,b] == 5)
                    j_5 = (c_single_genotype_2[a,b] == 5)
                    k_5 = (a == 5)
                    if i_4 or j_4 or k_4 or i_5 or j_5 or k_5:
                        #print("mut is added")
                        mut += 1
        mut_counter.append(mut)
    return mut_counter
    
# a function to convert style of a given genotype
# from matrix with style of(N,N*rho) to matrix of(3,something)
def convert_format_genotype(single_genotype_1, single_genotype_2):
    new_genotype = np.zeros((int(N*N*rho),3))
    counter = 0
    for a in range(N):
        for b in range(int(N*rho)):
            k = a
            i = single_genotype_1[a, b]
            j = single_genotype_2[a, b]
            new_genotype[counter,0] = i
            new_genotype[counter,1] = j
            new_genotype[counter,2] = k
            counter += 1
    return new_genotype

    
##second block calculating x


def get_dx(x, times, single_genotype_1, single_genotype_2, environment):      #DE for one network
  #calculate the tensor sum of the network and x
  #single_genotype_1 has the structure of [N, N * rho]
  
    R = np.zeros(N)
    for k in range(N):
        for num in range(int(N*rho)):
            #print(k,num)
            #print(single_genotype_2[k, num])
            #print(int(single_genotype_2[k, num]))
            R[int(single_genotype_1[k, num])] += x[int(single_genotype_2[k, num])] * x[k]
            #print(x[int(single_genotype_2[k, num])],"k=",k,"num=",num,x[k])
            #print("i=",int(single_genotype_1[k, num]), "j=",int(single_genotype_2[k, num]))
            #ちゃんとgenotypeも変わってる
            R[int(single_genotype_2[k, num])] -= x[int(single_genotype_2[k, num])] * x[k]

    dx = np.zeros(N)
    for a in range(N):
        if a < n:
            dx[a] = R[a] + D * environment[a] * x[a + n] - fitness(environment, x) * x[a]
        else:
            dx[a] = R[a] - fitness(environment, x) * x[a]
    return dx


#giving slight change of the network, change a part of network to 0.9 (for analysis)
def get_dx_2(x, times, single_genotype_1, single_genotype_2, environment,random_string):
  #calculate the tensor sum of the network and x
  #single_genotype_1 has the structure of [N, N * rho]
  #print("0")
  sum = np.sum(x)
  x = x / sum
  
  R = np.zeros(N)
  for k in range(N):
    for num in range(int(N*rho)):
        
        if N*num + k in random_string:
            kappa = 0.99
            #print("kappa=0.9","[",str(N*num+k),"]")
        else:
            kappa = 1
        
        R[int(single_genotype_1[k, num])] += kappa * x[int(single_genotype_2[k, num])] * x[k]
        R[int(single_genotype_2[k, num])] -= kappa * x[int(single_genotype_2[k, num])] * x[k]

  dx = np.zeros(N)
  for a in range(N):
    if a < n:
      dx[a] = R[a] + D * environment[a] * x[a + n] - fitness(environment, x) * x[a]
    else:
      dx[a] = R[a] - fitness(environment, x) * x[a]
  return dx



#if g = True, graph is drawn
def calc_x(get_dx, times, single_genotype_1, single_genotype_2, environment, g):
  #print(single_genotype)
  #print("a")
  init_x = np.ones(N) / N
  sol = odeint(get_dx, init_x, times, args = (single_genotype_1,single_genotype_2, environment,))
  final_dx = get_dx(sol[len(times)-1], times, single_genotype_1, single_genotype_2, environment)
  ## i guess if dx is less than 1e-6, it can be regarded as stationary
  #print("final_dx = ", final_dx)
  if g:
        plt.plot(times, sol)
        plt.plot(times, sol[:,0],label = "gene 1")
        plt.plot(times, sol[:,4],label = "transporter 1")
        plt.plot(times, sol[:,5],label = "transporter 2")
        plt.legend()
        plt.savefig("graph of chemical", dpi = 300, format = "png")
        plt.show()

  #for i in range(int(n/2):
    #plt.plot(times, sol[:,n+i],label = "gene " + str(i))

  #plt.legend()
  #plt.show()
  stat_value = sol[len(times)-1,:]
  #print(stat_value.shape)
  return stat_value, final_dx

def calc_x_2(get_dx_2, times, single_genotype_1, single_genotype_2, environment, random_string, g):
  #print(single_genotype)
  #print("a")
  init_x = np.ones(N) / N
  sol = odeint(get_dx_2, init_x, times, args = (single_genotype_1,single_genotype_2, environment, random_string, ))
  final_dx = get_dx_2(sol[len(times)-1], times, single_genotype_1, single_genotype_2, environment, random_string, )
  ## i guess if dx is less than 1e-6, it can be regarded as stationary
  #print("final_dx = ", final_dx)
  if g:
        plt.plot(times, sol)
        plt.plot(times, sol[:,0],label = "gene 1")
        plt.plot(times, sol[:,4],label = "transporter 1")
        plt.plot(times, sol[:,5],label = "transporter 2")
        plt.legend()
        plt.savefig("graph of chemical", dpi = 300, format = "png")
        plt.show()

  #for i in range(int(n/2):
    #plt.plot(times, sol[:,n+i],label = "gene " + str(i))

  #plt.legend()
  #plt.show()
  stat_value = sol[len(times)-1,:]
  #print(stat_value.shape)
  return stat_value, final_dx


##third block evaluating networks
def fitness(environment, x):
  fitness = 0
  for i in range(n):
    fitness += D * environment[i] * x[n+i]
    #print("adding environment["+str(i)+"]+x["+str(n+i))
    #print("the value is, env"+str(environment[i])+",x"+str(x[n+i]))
    #print(fitness)
  return fitness
  
def Lselect(genotype_1, genotype_2, times, environment, L,g):
  fitness_matrix = np.zeros(c*L)
  parents_1 = np.zeros((L,N, int(N*rho)))
  parents_2 = np.zeros((L,N, int(N*rho)))
  t_stat_value_matrix = np.zeros((c*L, N))      #a matrix for storing the stationary values of x for PCA (this is temporal)
  stat_value_matrix = np.zeros((L, N))
  for i in range(c*L):
    stat_value, final_dx = calc_x(get_dx, times, genotype_1[i,:,:],genotype_2[i,:,:], environment, g)
    #print("phenotype of the transporters are, ", stat_value[n:2*n])
    #print(i)
    t_stat_value_matrix[i,:] = stat_value
    fitness_matrix[i] = fitness(environment, stat_value)
    if np.sum(abs(final_dx)) > 1e-6 * N:
      fitness_matrix[i] = 0 
    #print(fitness(environment, stat_value))
  #print("fitness is", fitness_matrix)
  arranged_array = np.argsort(fitness_matrix)   
  index_of_top_L = arranged_array[c*L-L:]
  #index_of_bottom = arranged_array[:c*L-L]
  for k in range(L):
    stat_value_matrix[k] = t_stat_value_matrix[index_of_top_L[k]]  #####check later
  #print(index_of_top_L)
  #print(arranged_array)
  for j in range(L):
    parents_1[j,:,:] = genotype_1[index_of_top_L[j], :,:]
    parents_2[j,:,:] = genotype_2[index_of_top_L[j], :,:]
  return parents_1, parents_2, index_of_top_L, fitness_matrix, stat_value_matrix


#identifier is the string which is used for distinguish the trial
def evolution(genotype_1, genotype_2, G, environment,identifier):  
    top_matrix = []
    whole_stat_value_matrix = np.zeros((L*G,N))
    for i in range(G):
        g = False
        if (i+1) % div == 0:
            g=True
        print("generation", i)
        parents_1, parents_2, index_of_top_L, fitness_matrix, stat_value_matrix = Lselect(genotype_1,genotype_2, times, environment, L,g)
        #print(parents_1[0,:,:])
        if (i+1) % div == 0:
            savetxt(str(i+1) + 'th_genotype1' +identifier+ '.csv', parents_1[0,:,:], delimiter=',')   #to save numpy array with savetxt,
            savetxt(str(i+1) + 'th_genotype2' +identifier+ '.csv', parents_2[0,:,:], delimiter=',')   #the shape should be (m,n) not, (m,n,l)
            savetxt(str(i+1) + 'th_phenptype' +identifier+ '.csv', whole_stat_value_matrix, delimiter=',')
            savetxt(str(i+1) + 'th_graph_of_fitness' +identifier+ '.csv', top_matrix, delimiter=',')  #fitness 偶にめっちゃ変わってるから、入れときゃ良かった泣
            plt.plot(top_matrix, '.')
            plt.title("change of fitness N=" + str(N) + ",G=" + str(G)+",t_max ="+str(t_max)+",myu ="+str(myu)+",n ="+str(n)+",D ="+str(D)+",L ="+str(L)+",c ="+str(c)+",rho ="+str(rho))
            plt.savefig(identifier + "change of fitness N=" + str(N) + ",G=" + str(G)+",t_max ="+str(t_max)+",myu ="+str(myu)+",n ="+str(n)+",D ="+str(D)+",L ="+str(L)+",c ="+str(c)+",rho ="+str(rho), dpi = 300, format = 'png')
            plt.show()
            savetxt(str(i+1) + 'th_whole_phenotype****.csv', whole_stat_value_matrix, delimiter=',')
            generation = np.arange(0,i)
            for j in range(int(n/2)):
                plt.plot(generation, whole_stat_value_matrix[generation,n+j])
                plt.title("change of transporter")
                plt.show()

        whole_stat_value_matrix[i,:] = stat_value_matrix[0,:]      #phenotype of top L networks-> phenotype of top network

        top_index = index_of_top_L[L-1]
        top_matrix.append(fitness_matrix[top_index])
    
        genotype_1, genotype_2, change_genotype_1, change_genotype_2 = mutate_network(parents_1, parents_2, N, c, L, myu)

    return parents_1[0,:,:], parents_2[0,:,:], whole_stat_value_matrix
  

    
#紛らわしいから、番号はexcelに合わせる
def calc_of_restored(genotype_1,genotype_2, change_of_whole_1, change_of_whole_2, genotype_of_interest):
    change_of_interest_1 = change_of_whole_1[int(genotype_of_interest) - 1]
    #print(change_of_interest_1)
    change_of_interest_2 = change_of_whole_2[int(genotype_of_interest) - 1]
    
    genotype_1, genotype_2 = restore_genotype(genotype_1,genotype_2,change_of_interest_1,change_of_interest_2,check = 0)
    stat_value, final_dx = calc_x(get_dx, times, genotype_1, genotype_2, environment, g=True)
    #savetxt('genotype_1,'+str(genotype_of_interest)+'.csv', genotype_1, delimiter=',')
    #savetxt('genotype_2,'+str(genotype_of_interest)+'.csv', genotype_2, delimiter=',')
    
    return genotype_1, genotype_2, stat_value, final_dx

#mutate_for_type1 からの
def calc_of_restored_2(genotype_1,genotype_2, change_of_whole, genotype_of_interest):
    change_new = np.zeros((M, 32))
    change_new = change_of_whole
    reshaped = change_of_new.reshape([M,16,2])
    change_of_interest = reshaped[int(genotype_of_interest) - 1, :,:]
    random_string = []
    for i in range(16):
        random_string.append(change_of_interest[i,0]* N + change_of_interest[i,1])
    
    stat_value, final_dx = calc_x_2(get_dx, times, genotype_1, genotype_2, environment, random_string, g=True)
    #savetxt('genotype_1,'+str(genotype_of_interest)+'.csv', genotype_1, delimiter=',')
    #savetxt('genotype_2,'+str(genotype_of_interest)+'.csv', genotype_2, delimiter=',')
    
    return genotype_1, genotype_2, stat_value, final_dx

    
def PCA_analysis(genotype_1, genotype_2, N, M, rho, myu, mut_type, trial_number):
    evolved_genotype_1 = np.zeros((1,N,int(N*rho)))
    evolved_genotype_2 = np.zeros((1,N,int(N*rho)))
    evolved_genotype_1[0,:,:] = genotype_1
    evolved_genotype_2[0,:,:] = genotype_2
    mutated_evolved_phenotype = np.zeros((M,N)) #inverse the index temporarily

    r_counter = 0
    e_counter = 0
    # the code below is to execute the creation of mutated networks by the same mutation algorithm as that during the evolution
    if mut_type == 0:
        

        print(evolved_genotype_1[0,0,0])

        mutated_evolved_phenotype = np.zeros((M,N)) #inverse the index temporarily

        r_counter = 0
        e_counter = 0
        trial_number = "*"

        change_of_whole_1 = []
        change_of_whole_2 = []
        length_1 = []
        length_2 = []
        evolved_fitness = []
        for i in range(M):

            if e_counter % div == 0:
                print(r_counter, e_counter)

            mutated_evolved_genotype_1 = np.zeros((N,int(N*rho)))
            mutated_evolved_genotype_2 = np.zeros((N,int(N*rho)))
            change_evolved_genotype_1 = []
            change_evolved_genotype_2 = []

            if i == 0:
                mutated_evolved_genotype_1 = evolved_genotype_1[0,:,:]
                mutated_evolved_genotype_2 = evolved_genotype_2[0,:,:]
            else:
                mutated_evolved_genotype_1, mutated_evolved_genotype_2,change_evolved_genotype_1, change_evolved_genotype_2 = mutate_network(evolved_genotype_1, evolved_genotype_2, N, 2, 1, myu)   #random_genotypeになってた...orz
                mutated_evolved_genotype_1, mutated_evolved_genotype_2 = mutated_evolved_genotype_1[1,:,:], mutated_evolved_genotype_2[1,:,:]
            change_of_single_1 = list(itertools.chain.from_iterable(change_evolved_genotype_1)) #to flatten the change_matrix
            change_of_single_2 = list(itertools.chain.from_iterable(change_evolved_genotype_2))
            length_1.append(len(change_of_single_1))
            length_2.append(len(change_of_single_2))
            change_of_whole_1.append(change_of_single_1)    ###append の引数はNoneを返す
            change_of_whole_2.append(change_of_single_2)

            e_stat_value, e_final_dx = calc_x(get_dx, times, mutated_evolved_genotype_1, mutated_evolved_genotype_2, environment, g=False)
            evolved_fitness.append(fitness(environment, e_stat_value))
            mutated_evolved_phenotype[i,:] = e_stat_value
            if abs(np.sum(e_final_dx)) < 1e-6 * N:
                e_counter += 1
        print("counter is,", r_counter, e_counter)
        print("length_1 is", length_1)

        max_1 = max(length_1)
        max_2 = max(length_2)

        #fill the list with paddings
        for i in range(len(change_of_whole_1)):
            padding = [400 for i in range(max_1 - len(change_of_whole_1[i]))]
            change_of_whole_1[i] = change_of_whole_1[i]+padding

        for i in range(len(change_of_whole_2)):
            padding = [400 for i in range(max_2 - len(change_of_whole_2[i]))]
            change_of_whole_2[i] = change_of_whole_2[i]+padding

        print(change_of_whole_1)
        savetxt(str(trial_number)+'evolvedphenotype(N='+str(N)+',G='+str(G)+',L=' + str(L) + 'M='+str(M) + ').csv', mutated_evolved_phenotype, delimiter=',')
        savetxt(str(trial_number)+' evolved change_of_genotype_1,G='+str(G)+'M='+str(M)+').csv', change_of_whole_1, delimiter=',')
        savetxt(str(trial_number)+' evolved change_of_genotype_2,G='+str(G)+'M='+str(M)+').csv', change_of_whole_2, delimiter=',')
        savetxt(str(trial_number)+'evolved_fitness.csv', evolved_fitness, delimiter=',')


    ##change_of_genotypeでは最初は保存されないので、そこらへんの帳尻合わせに気をつける

            #the code below is to analyse the data with the different mutation method
    #here, random numbers are generated and reaction coefficient is reduced accordingly
    if mut_type == 1:
        change_of_whole = []
        evolved_fitness = []
        mutated_evolved_genotype_1 = evolved_genotype_1[0,:,:]
        mutated_evolved_genotype_2 = evolved_genotype_2[0,:,:]
        e_counter = 0
        for i in range(M):
            if e_counter % div == 0:
                print(e_counter)
            if i ==0:
                print("calc_x is executed")
                e_stat_value, e_final_dx = calc_x(get_dx, times, mutated_evolved_genotype_1, mutated_evolved_genotype_2, environment, g=True)
                string_for_save = [0] * 8
                #TypeError: 'int' object is not iterable
            else:
                string_for_save = []
                random_string, string_for_save = mutate_for_type1(int(N*myu))
                e_stat_value, e_final_dx = calc_x_2(get_dx_2, times, mutated_evolved_genotype_1, mutated_evolved_genotype_2, environment, random_string, g=False)
                string_for_save = list(itertools.chain.from_iterable(string_for_save)) #to flatten the change_matrix

                
                #print("phenotype",mutated_evolved_phenotype)
            #print("before folded",string_for_save)
            mutated_evolved_phenotype[i,:] = e_stat_value
            change_of_whole.append(string_for_save)
            evolved_fitness.append(fitness(environment, e_stat_value))
            #print(evolved_fitness)
            #print(i,change_of_whole)
            e_counter += 1
        
        print("the change is", change_of_whole)
        print("shape of the change is",np.array(change_of_whole).shape)
        savetxt(str(trial_number)+'evolvedphenotype(N='+str(N)+',G='+str(G)+',L=' + str(L) + 'M='+str(M) + ').csv', mutated_evolved_phenotype, delimiter=',')
        savetxt(str(trial_number)+'evolved change_of_genotype,G='+str(G)+'M='+str(M)+').csv', np.array(change_of_whole), delimiter=',')
        savetxt(str(trial_number)+'evolved_fitness.csv', np.array(evolved_fitness), delimiter=',')


    ##change_of_genotypeでは最初は保存されないので、そこらへんの帳尻合わせに気をつける

    evolved_pca = PCA(n_components=N)
    evolved_pca.fit(mutated_evolved_phenotype)
    Ve = evolved_pca.explained_variance_ratio_
    Xe = evolved_pca.transform(mutated_evolved_phenotype)
    plt.plot(Xe[:,0],Xe[:,1],".")
    evolved_2_components=np.zeros((M,2))  #どっちのindexかはちゃんと確認しておかんと...
    evolved_2_components[:,0] = Xe[:,0]
    evolved_2_components[:,1] = Xe[:,1]
    savetxt(str(trial_number)+'evolved_2_PC.csv', evolved_2_components, delimiter=',')
    loadings = evolved_pca.components_.T * np.sqrt(evolved_pca.explained_variance_)
    savetxt(str(trial_number)+"_"+str(G)+'loadings.csv', loadings, delimiter=',')
    print("loadings is", loadings)
    print("shape of componenets is", evolved_pca.components_.shape)
    print("components are", evolved_pca.components_)
    savetxt(str(trial_number)+"_"+str(G)+'evolved eigen vectors.csv', evolved_pca.components_, delimiter=',')
    #savetxt('2_PC throughout evolution.csv', evolved_2_components, delimiter=',')
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.title("PCA of evolved network")
    plt.text(0.0001,0.0013,"r(fitness - PC1) = "+str(np.corrcoef(Xe[:,0], evolved_fitness)[0,1]))
    plt.text(0.0001,0.001,"r(fitness - PC2) = "+str(np.corrcoef(Xe[:,1], evolved_fitness)[0,1]))
    plt.savefig(str(trial_number)+"PCA of evolved network", dpi=300, format="png")
    plt.show()


    #plt.plot(np.arange(n_components), Vr, linestyle='--', marker='o', label="random network")
    plt.plot(np.arange(n_components), Ve, linestyle='--', marker='o', label="evolved network")
    plt.legend()
    plt.title("explained variance ratio")
    plt.xlabel("Index of PCA")
    plt.ylabel("Explained variance ratio")
    plt.savefig(str(trial_number)+"evolved Explained variance ratio", dpi=300, format="png")
    savetxt(str(trial_number)+'evolved explained_variance_ratio.csv', Ve, delimiter=',')
    plt.show()



In [ ]:
%%time
l = [x ** 2 for x in range(100000)]
l = [x ** 2 for x in range(100000)]
l = [x ** 2 for x in range(100000)]
l = [x ** 2 for x in range(100000)]
l = [x ** 2 for x in range(100000)]
l = [x ** 2 for x in range(100000)]

##最初から進化させる


N= 400 #化学物質の数
n = 10     #nutrient・transporterの数
D = 1/N #拡散係数
rho = 0.04   #各化学物質あたりの反応経路の数
L = 10 #各世代で選ばれる細胞数
c = 10 #コピー数
myu = 0.04 #突然変異率
G = 3000
#世代数
div = 50 #saveする間隔

identifier = "*5"
environment = np.array([1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]) / 5

evolved_genotype_1 = np.zeros((c*L, N, int(N*rho)))
evolved_genotype_2 = np.zeros((c*L, N, int(N*rho)))

for i in range(c*L):
    evolved_genotype_1[i,:,:] = np.genfromtxt("3000th_genotype1.csv" ,delimiter=",")
    evolved_genotype_2[i,:,:] = np.genfromtxt("3000th_genotype2.csv" ,delimiter=",")

print(environment)
evolved_genotype_1, evolved_genotype_2, whole_stat_value_matrix = evolution(evolved_genotype_1, evolved_genotype_2, 1000, environment, identifier)

#save evolved_genotype_1, evolved_genotype_2
savetxt('genotype1,G='+str(G)+')'+identifier+ '.csv', evolved_genotype_1, delimiter=',')
savetxt('genotype2,G='+str(G)+')'+identifier+ '.csv', evolved_genotype_2, delimiter=',')
savetxt('phenotype,G='+str(G)+')'+identifier+ '.csv', whole_stat_value_matrix, delimiter=',')


In [ ]:
##途中から進化を再開する
##a function to start evolution from a preserved genotype

N= 400 #化学物質の数
n = 10     #nutrient・transporterの数
D = 1/N #拡散係数
rho = 0.04   #各化学物質あたりの反応経路の数
L = 10 #各世代で選ばれる細胞数
c = 10 #コピー数
myu = 0.04 #突然変異率
G = 3000
#世代数
div = 10 #saveする間隔

identifier = "*5"
environment = np.array([1.,1.,1.,1.,1.,0.,0.,0.,0.,0.]) / 5

initial_genotype_1 = np.zeros((c*L, N, int(N*rho)))
initial_genotype_2 = np.zeros((c*L, N, int(N*rho)))

#for i in range(c*L):
#    evolved_genotype_1[i,:,:] = np.genfromtxt("3000th_genotype1.csv" ,delimiter=",")
#    evolved_genotype_2[i,:,:] = np.genfromtxt("3000th_genotype2.csv" ,delimiter=",")

initial_genotype_1, initial_genotype_2 = generate_genotype(c, L, N, rho, trial_number=0)
print(environment)
evolved_genotype_1, evolved_genotype_2, whole_stat_value_matrix = evolution(initial_genotype_1, initial_genotype_2, G, environment, identifier)

#save evolved_genotype_1, evolved_genotype_2
savetxt('genotype1,G='+str(G)+')'+identifier+ '.csv', evolved_genotype_1, delimiter=',')
savetxt('genotype2,G='+str(G)+')'+identifier+ '.csv', evolved_genotype_2, delimiter=',')
savetxt('phenotype,G='+str(G)+')'+identifier+ '.csv', whole_stat_value_matrix, delimiter=',')


In [ ]:
%%time
l = [x ** 2 for x in range(100000)]
l = [x ** 2 for x in range(100000)]
l = [x ** 2 for x in range(100000)]
l = [x ** 2 for x in range(100000)]
l = [x ** 2 for x in range(100000)]
l = [x ** 2 for x in range(100000)]

##A program to get PC structure after introducing the environmental change
#firstly, get the random genotype and evolved genotype
#secondly, introduce the mutations on the genotype by mutation function
#Thirdly, by calc_x function, get the stationary value (phenotype)
#Fourthly, for obtained stationary value, make a large matrix that contains all the phenotype

import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from numpy import savetxt
from sklearn.decomposition import PCA
import itertools

print("started")
    
G = 3000
M = 3000
myu = 0.04
div = 100 #counterをprintする間隔
dt=0.1
t_max=600                     
times=np.arange(0,t_max,dt)
n_components = N
environment = np.array([1.,1.,0.,0.])

evolved_genotype_1 = np.genfromtxt("3000th_genotype1.csv" ,delimiter=",")
evolved_genotype_2 = np.genfromtxt("3000th_genotype2.csv" ,delimiter=",")
#random_genotype_1, random_genotype_2 = generate_genotype(1, 1, N, rho)

print(evolved_genotype_1)

#genotype2もprintしてたら異変に気づけたのね...
PCA_analysis(evolved_genotype_1, evolved_genotype_2, N, M, rho, myu/4, mut_type=1, trial_number=22)

In [ ]:
# importしたphenotypeを解析するプログラム
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from numpy import savetxt
from sklearn.decomposition import PCA

print("started")
 
G = 3000
M = 3000
div = 100 #counterをprintする間隔
dt=0.1
t_max=600                     
times=np.arange(0,t_max,dt)
n_components = N
environment = np.array([1.,1.,0.,0.])
trial_number = "16"

#change_of_whole_1 = np.genfromtxt("10 evolved change_of_genotype_1,G=3000M=3000).csv" ,delimiter=",")
#change_of_whole_2 = np.genfromtxt("10 evolved change_of_genotype_2,G=3000M=3000).csv" ,delimiter=",")
genotype_1 = np.genfromtxt("3000th_genotype1.csv" ,delimiter=",")
genotype_2 = np.genfromtxt("3000th_genotype2.csv" ,delimiter=",")

mutated_evolved_phenotype = np.genfromtxt("16evolvedphenotype(N=400,G=3000,L=10M=3000).csv", delimiter=",")
#print(mutated_evolved_phenotype)
#fitness = np.genfromtxt("9random_fitness.csv", delimiter=",")

evolved_fitness = []
for i in range(M):
    #x = mutated_evolved_phenotype[i,:]
    #print(np.array(x))
    #print(fitness(environment, np.array(x)))
    evolved_fitness.append(fitness(environment, mutated_evolved_phenotype[i,:]))
    
print(len(evolved_fitness))
#evolved_fitness = np.array(evolved_fitness)
#print(evolved_fitness.shape)
#savetxt('7evolved_fitness.csv', evolved_fitness, delimiter=',')

evolved_pca = PCA(n_components=N)
evolved_pca.fit(mutated_evolved_phenotype)
Ve = evolved_pca.explained_variance_ratio_
Xe = evolved_pca.transform(mutated_evolved_phenotype)
evolved_2_components=np.zeros((M,2))  #どっちのindexかはちゃんと確認しておかんと...
evolved_2_components[:,0] = Xe[:,0]
evolved_2_components[:,1] = Xe[:,1]

###code block to pick phenotypes having abnormally large PC
#then draw the graph to see whether all values become stationary
###pick the phenotype with reference to PC1
index_pc1 = np.argsort(Xe[:,0])
print("top PC1 is", Xe[-1,0])
print("the lowest PC1 is", Xe[0,0])
index_pc1 = index_pc1[0:4]
print(index_pc1)
index_pc1 = index_pc1 + 1  #to make the numbers same as those of excel's
print(index_pc1)
index_pc2 = np.argsort(Xe[:,1])
index_pc2 = index_pc2[0:4]
index_pc2 = index_pc2 + 1  #to make the numbers same as those of excel's

"""
for i in range(len(index_pc1)):
    calc_of_restored(genotype_1,genotype_2, change_of_whole_1, change_of_whole_2, index_pc1[i])
for i in range(len(index_pc2)):
    calc_of_restored(genotype_1,genotype_2, change_of_whole_1, change_of_whole_2, index_pc2[i])
"""  

plt.plot(Xe[:,0],Xe[:,1],".")
#hist_plot(Xe[:,0],Xe[:,1])
savetxt(trial_number+'evolved_2_PC.csv', evolved_2_components, delimiter=',')
loadings = evolved_pca.components_.T * np.sqrt(evolved_pca.explained_variance_)
#savetxt("9"+str(G)+'loadings.csv', loadings, delimiter=',')
#print("loadings is", loadings)
#print("shape of componenets is", evolved_pca.components_.shape)
#print("components are", evolved_pca.components_)
savetxt(trial_number+str(G)+'eigen vectors.csv', evolved_pca.components_, delimiter=',')
#savetxt('2_PC throughout evolution.csv', evolved_2_components, delimiter=',')
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PCA of evolvednetwork")
plt.text(0.0001,0.0005,"r(fitness - PC1) = "+str(np.corrcoef(Xe[:,0], evolved_fitness)[0,1]))
plt.text(0.0001,0.0004,"r(fitness - PC2) = "+str(np.corrcoef(Xe[:,1], evolved_fitness)[0,1]))
plt.savefig(trial_number+" PCA of evolved network(3)", dpi=300, format="png")
plt.show()


plt.plot(np.arange(n_components), Ve, linestyle='--', marker='o', label="evolved network")
plt.legend()
plt.title("explained variance ratio")
plt.xlabel("Index of PCA")
plt.ylabel("Explained variance ratio")
plt.savefig(trial_number+" explained_variance_ratio", dpi=300, format="png")
savetxt(trial_number+str(G)+'explained_variance_ratio throughout evolution.csv', Ve, delimiter=',')
plt.show()

In [ ]:
##the cell to see the time development of chemicals from restored genotype
change_of_whole_1 = np.genfromtxt("4change_of_genotype_1,G=3000M=3000).csv" ,delimiter=",")
change_of_whole_2 = np.genfromtxt("4change_of_genotype_2,G=3000M=3000).csv" ,delimiter=",")
genotype_1 = np.genfromtxt("3000th_genotype1.csv" ,delimiter=",")
genotype_2 = np.genfromtxt("3000th_genotype2.csv" ,delimiter=",")

#savetxt('original_converted_genotype.csv', new_genotype, delimiter=',')


#紛らわしいから、番号はexcelに合わせる
def calc_of_restored(genotype_1,genotype_2, change_of_whole_1, change_of_whole_2, genotype_of_interest):
    change_of_interest_1 = change_of_whole_1[int(genotype_of_interest) - 1]
    #print(change_of_interest_1)
    change_of_interest_2 = change_of_whole_2[int(genotype_of_interest) - 1]
    
    genotype_1, genotype_2 = restore_genotype(genotype_1,genotype_2,change_of_interest_1,change_of_interest_2,check = 0)
    stat_value, final_dx = calc_x(get_dx, times, genotype_1, genotype_2, environment)
    #savetxt('genotype_1,'+str(genotype_of_interest)+'.csv', genotype_1, delimiter=',')
    #savetxt('genotype_2,'+str(genotype_of_interest)+'.csv', genotype_2, delimiter=',')
    
    return genotype_1, genotype_2, stat_value, final_dx

##番号がなんだかんだ合ってた...じゃないズレてた（changeのデータセットにoriginalが入ってなかった（-1）, indexの問題(-1)）

##interestは=1のとき、original...の予定だった
#いや、一つ目は間違えてmutationのデータが入ってた
#mutateした後に、original genotypeを入れてた
genotype_of_interest = 1
genotype_1, genotype_2, stat_value, final_dx = calc_of_restored(genotype_1,genotype_2, change_of_whole_1, change_of_whole_2,genotype_of_interest)

new_genotype = convert_format_genotype(genotype_1, genotype_2)
savetxt('converted_genotype,'+str(genotype_of_interest)+'.csv', new_genotype, delimiter=',')



print(genotype_1)
print(genotype_2)

In [ ]:
##program to get the indices of reactions which invloved in the production/consumption/catalysis of a given chemical

def get_indices(single_genotype_1, single_genotype_2, chemical):
    indices = []
    for a in range(N):
        for b in range(int(N*rho)):
            i = single_genotype_1[a,b]
            j = single_genotype_2[a,b]
            k = a
            if i == chemical or j == chemical or k == chemical:
                indices.append([i,j,k])
    return indices

#count the number of reactions of each chemical
def count_indices(single_genotype_1, single_genotype_2):
    count_matrix = np.zeros((N,2))
    for i in range(N):
        chemical_matrix = get_indices(single_genotype_1, single_genotype_2, i)
        reaction_counter = len(chemical_matrix)
        count_matrix[i,0] = i+1
        count_matrix[i,1] = reaction_counter
    return count_matrix
        
    

evolved_genotype_1 = np.genfromtxt("3000th_genotype1.csv" ,delimiter=",")
evolved_genotype_2 = np.genfromtxt("3000th_genotype2.csv" ,delimiter=",")

chemical_number = 5
indices = get_indices(evolved_genotype_1, evolved_genotype_2, chemical_number-1)
print(indices)
print(len(indices))
savetxt("indices for"+str(chemical_number)+",G="+str(G)+".csv",indices, delimiter=',')

count_matrix = count_indices(evolved_genotype_1, evolved_genotype_2)
#savetxt("count of reactions for,G="+str(G)+".csv",count_matrix, delimiter=',')